# Notes to use SAR on MovieLens (Python, CPU)

SAR is a fast scalable adaptive algorithm for personalized recommendations based on user transaction history. It produces easily explainable / interpretable recommendations and handles "cold item" and "semi-cold user" scenarios. SAR is a kind of neighborhood based algorithm (as discussed in [Recommender Systems by Aggarwal](https://dl.acm.org/citation.cfm?id=2931100)) which is intended for ranking top items for each user. 

SAR recommends items that are most ***similar*** to the ones that the user already has an existing ***affinity*** for. Two items are ***similar*** if the users who have interacted with one item are also likely to have interacted with another. A user has an ***affinity*** to an item if they have interacted with it in the past.

### SAR has the following advantages:
- High accuracy for an easy to train and deploy algorithm
- Fast training, only requiring simple counting to construct matrices used at prediction time. 
- Fast scoring, only involving multiplication of the similarity matric with an affinity vector

### And the following drawbacks:
- Since it does not use item or user features, it can be at a disadvantage against algorithms that do.
- It's memory-hungry, requiring the creation of an $mxm$ sparse square matrix (where $m$ is the number of items). This can also be a problem for many matrix factorization algorithms.
- SAR favors an implicit rating scenario and it does not predict ratings.

This notebook provides an example of how to utilize and evaluate SAR in Python on a CPU.

# 0 Global Settings and Imports

In [3]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")

import os
from zipfile import ZipFile

from reco_utils.recommender.sar.sar_singlenode import SARSingleNodeReference
from reco_utils.dataset.url_utils import maybe_download
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

import itertools
import pandas as pd

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.6.0 | packaged by conda-forge | (default, Feb  9 2017, 14:36:55) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
Pandas version: 0.23.4


# 1 Load Data

SAR is intended to be used on interactions with the following schema:
`<User ID>, <Item ID>,<Time>,[<Event Type>], [<Event Weight>]`. 

Each row represents a single interaction between a user and an item. These interactions might be different types of events on an e-commerce website, such as a user clicking to view an item, adding it to a shopping basket, following a recommendation link, and so on. Each event type can be assigned a different weight, for example, we might assign a “buy” event a weight of 10, while a “view” event might only have a weight of 1.

The MovieLens dataset is well formatted interactions of Users providing Ratings to Movies (movie ratings are used as the event weight) - we will use it for the rest of the example.

### 1.1 Download and use the MovieLens Dataset

In [4]:
# 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '10m'

In [5]:
# MovieLens data have different data-format for each size of dataset 
data_header = None
if MOVIELENS_DATA_SIZE == '100k':
    separator = '\t'
    data_name = 'u.data'
    data_folder = 'ml-100k'
elif MOVIELENS_DATA_SIZE == '1m':
    separator = '::'
    data_name = 'ratings.dat'
    data_folder = 'ml-1m'
elif MOVIELENS_DATA_SIZE == '10m':
    separator = '::'
    data_name = 'ratings.dat'
    data_folder = 'ml-10M100K'
elif MOVIELENS_DATA_SIZE == '20m':
    separator = ','
    data_name = 'ratings.csv'
    data_folder = 'ml-20m'
    data_header = 0
else:
    raise ValueError('Invalid data size. Should be one of {100k, 1m, 10m, or 20m}') 

# Download dataset zip file and decompress if haven't done yet
filename = 'ml-' + MOVIELENS_DATA_SIZE + '.zip'
filepath = maybe_download('http://files.grouplens.org/datasets/movielens/'+filename, filename)

data_path = os.path.join(data_folder, data_name)
if not os.path.exists(data_path):
    with ZipFile(filepath, 'r') as zf:
        zf.extractall()
    
data = pd.read_csv(
    data_path,
    sep=separator,
    engine='python',
    names=['UserId','MovieId','Rating','Timestamp'],
    header=data_header
)
data.head()

,UserId,MovieId,Rating,Timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


### 1.2 Split the data using the python random splitter provided in utilities:

We utilize the provided `python_random_split` function to split into `train` and `test` datasets randomly at a 80/20 ratio.

In [6]:
train, test = python_random_split(data)

# 2 Train the SAR Model

### 2.1 Instantiate the SAR algorithm and set the index

In order to use SAR, we need to hash users and items and set an index in order for SAR matrix operations to work. First we instantiate SAR using the reference implementation provided in `SARSingleNodeReference`:

In [7]:
header = {
        "col_user": "UserId",
        "col_item": "MovieId",
        "col_rating": "Rating",
        "col_timestamp": "Timestamp",
    }

model = SARSingleNodeReference(
                remove_seen=True, similarity_type="jaccard", 
                time_decay_coefficient=30, time_now=None, timedecay_formula=True, **header
            )

In [8]:
unique_users = data["UserId"].unique()
unique_items = data["MovieId"].unique()

We will hash users and items to smaller continuous space.
This is an ordered set - it's discrete, but contiguous.
This helps keep the matrices we keep in memory as small as possible.

In [9]:
enumerate_items_1, enumerate_items_2 = itertools.tee(enumerate(unique_items))
enumerate_users_1, enumerate_users_2 = itertools.tee(enumerate(unique_users))
item_map_dict = {x: i for i, x in enumerate_items_1}
user_map_dict = {x: i for i, x in enumerate_users_1}

The reverse of the dictionary above - array index to actual ID


In [10]:
index2user = dict(enumerate_users_2)
index2item = dict(enumerate_items_2)

We need to index the train and test sets for SAR matrix operations to work

In [11]:
model.set_index(unique_users, unique_items, user_map_dict, item_map_dict, index2user, index2item)

### 2.2 Train the SAR model on our training data, and get the top-k recommendations for our testing data

SAR first computes an item-to-item ***co-occurence matrix***. Co-occurence represents the number of times two items appear together for any given user. Once we have the co-occurence matrix, we compute an ***item similarity matrix*** by rescaling the cooccurences by a given metric (Jaccard similarity in this example). 

We also compute an ***affinity matrix*** to capture the strength of the relationship between each user and each item. Affinity is driven by different types (like *rating* or *viewing* a movie), and by the time of the event. 

Recommendations are achieved by multiplying the affinity matrix $A$ and the similarity matrix $S$. The result is a ***recommendation score matrix*** $R$. We compute the ***top-k*** results for each user in the `recommend_k_items` function seen below.

A full walkthrough of the SAR algorithm can be found [here](../02_modeling/sar_educational_walkthrough.ipynb).

In [12]:
model.fit(train)
top_k = model.recommend_k_items(test)

../../reco_utils/recommender/sar/sar_singlenode.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["exponential"] = expo_fun(df[self.col_timestamp].values)
../../reco_utils/recommender/sar/sar_singlenode.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df["rating_exponential"] = df[self.col_rating] * df["exponential"]
../../reco_utils/recommender/sar/sar_singlenode.py:280: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.index = df.as_matrix([self._col_hashed_users, self._col_hashe

In [13]:
# TODO: remove this call when the model returns same type as input
top_k['UserId'] = pd.to_numeric(top_k['UserId'])
top_k['MovieId'] = pd.to_numeric(top_k['MovieId'])

In [14]:
display(top_k.head())

,UserId,MovieId,prediction
168329,60316,1265,201.683950
168328,60316,1682,200.576740
168327,60316,1036,198.259636
168326,60316,2716,194.903753
168325,60316,1089,191.972193


### 5. Evaluate how well SAR performs 

We evaluate how well SAR performs with for a few common ranking metrics provided in the `PythonRankingEvaluation` classs in utilities. We will consider the Mean Average Precision (MAP), Normalized Discounted Cumalative Gain (NDCG), Precision, and Recall for the top-k items per user we computed with SAR.

In [15]:
test.head()

,UserId,MovieId,Rating,Timestamp,hashedUsers
6326643,45172,2427,3.0,1123177433,44111
3046730,22038,3082,3.0,952140510,21465
4938457,35289,2946,3.0,1010090773,34441
4459548,31804,46970,3.0,1173245347,31014
6377100,45554,2858,3.5,1072293284,44487


In [16]:
k = 10

In [17]:
eval_map = map_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                    col_rating="Rating", col_prediction="prediction", 
                    relevancy_method="top_k", k=k)

In [18]:
eval_ndcg = ndcg_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                      col_rating="Rating", col_prediction="prediction", 
                      relevancy_method="top_k", k=k)

In [19]:
eval_precision = precision_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                                col_rating="Rating", col_prediction="prediction", 
                                relevancy_method="top_k", k=k)

In [20]:
eval_recall = recall_at_k(test, top_k, col_user="UserId", col_item="MovieId", 
                          col_rating="Rating", col_prediction="prediction", 
                          relevancy_method="top_k", k=k)

In [21]:
print("Model:\t" + model.model_str,
      "Top K:\t%d" % k,
      "MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

Model:	sar_ref
Top K:	10
MAP:	0.101402
NDCG:	0.321073
Precision@K:	0.275766
Recall@K:	0.156483
